In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
demandas = pd.read_csv("../tabelas_banco_csv/demandas.csv", encoding='latin-1')
usuarios = pd.read_csv("../tabelas_banco_csv/usuarios.csv", encoding='latin-1')
CCs = pd.read_csv("../tabelas_banco_csv/centrosDeCusto.csv", encoding='latin-1')
beneficiosDemanda = pd.read_csv("../tabelas_banco_csv/beneficiosDemanda.csv", encoding='latin-1')
CCsDemanda = pd.read_csv("../tabelas_banco_csv/centrosDeCustoDemanda.csv", encoding='latin-1')

In [3]:
demandas.head()

,id_demanda,frequencia_uso,objetivo,score,situacao_atual,titulo_demanda,id_usuario
0,1,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",1.7,Duis aute irure dolor in reprehenderit in volu...,Demanda 01,1
1,2,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",1.7,Duis aute irure dolor in reprehenderit in volu...,Demanda 02,1
2,3,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",56.0,Duis aute irure dolor in reprehenderit in volu...,Demanda 03,5
3,4,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",56.0,Duis aute irure dolor in reprehenderit in volu...,Demanda 04,5
4,11,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",12.0,Duis aute irure dolor in reprehenderit in volu...,Demanda Teste Histórico 01,2


In [4]:
usuarios.head()

,dtype,id_usuario,cargo,departamento,nome_usuario,numero_cadastro,setor
0,Solicitante,1,Técnico,TI,Diego,123,Suporte Técnico
1,Solicitante,2,Analista de Dados,TI,Eduarda,234,Segurança Dital
2,AnalistaTI,3,Desenvolvedor Backend,TI,João,345,Desenvolvimento de Software
3,GerenteNegocio,4,Desenvolvedor Frontend,TI,Camilly,456,Desenvolvimento de Software
4,Solicitante,5,Técnico,TI,Henrique,567,Administração de Redes e Sistemas Internos


In [5]:
#adicionando as informações de usuário na tabela de demanda
tipoSolicitante = []
cargoSolicitante = [] 
departamentoSolicitante = []
nomeSolicitante = []
setorSolicitante = []

for index, demanda in demandas.iterrows():
    usuario = usuarios.loc[usuarios['id_usuario'] == demanda.id_usuario]
    index = usuario.index[0]
    
    tipoSolicitante.append(usuarios.at[index, "dtype"])
    cargoSolicitante.append(usuario.at[index, "cargo"])
    departamentoSolicitante.append(usuario.at[index, "departamento"])
    nomeSolicitante.append(usuario.at[index, "nome_usuario"])
    setorSolicitante.append(usuario.at[index, "setor"])
    

demandas["tipo_solicitante"] = tipoSolicitante
demandas["cargo_solicitante"] = cargoSolicitante
demandas["departamento_solicitante"] = departamentoSolicitante
demandas["nome_solicitante"] = nomeSolicitante
demandas["setor_solicitante"] = setorSolicitante
    
demandas.head()

,id_demanda,frequencia_uso,objetivo,score,situacao_atual,titulo_demanda,id_usuario,tipo_solicitante,cargo_solicitante,departamento_solicitante,nome_solicitante,setor_solicitante
0,1,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",1.7,Duis aute irure dolor in reprehenderit in volu...,Demanda 01,1,Solicitante,Técnico,TI,Diego,Suporte Técnico
1,2,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",1.7,Duis aute irure dolor in reprehenderit in volu...,Demanda 02,1,Solicitante,Técnico,TI,Diego,Suporte Técnico
2,3,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",56.0,Duis aute irure dolor in reprehenderit in volu...,Demanda 03,5,Solicitante,Técnico,TI,Henrique,Administração de Redes e Sistemas Internos
3,4,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",56.0,Duis aute irure dolor in reprehenderit in volu...,Demanda 04,5,Solicitante,Técnico,TI,Henrique,Administração de Redes e Sistemas Internos
4,11,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",12.0,Duis aute irure dolor in reprehenderit in volu...,Demanda Teste Histórico 01,2,Solicitante,Analista de Dados,TI,Eduarda,Segurança Dital


In [6]:
demandas.drop(columns=["id_usuario"], inplace=True)

In [7]:
beneficiosDemanda.head()

,id_beneficio,descricao,moeda,tipo_beneficio,valor,id_demanda
0,1,Sit amet porttitor eget dolor morbi non arcu r...,NaN,QUALITATIVO,0.0,1
1,2,Sit amet porttitor eget dolor morbi non arcu r...,NaN,QUALITATIVO,0.0,1
2,3,Sit amet porttitor eget dolor morbi non arcu r...,NaN,QUALITATIVO,0.0,2
3,4,Sit amet porttitor eget dolor morbi non arcu r...,NaN,QUALITATIVO,0.0,2
4,5,Sit amet porttitor eget dolor morbi non arcu r...,REAL,POTENCIAL,78.0,3


In [8]:
def converter_valor(valor_puro, moeda):
    cotacao = 1
    
    if moeda == "DOLAR":
        cotacao = 4.78
    elif moeda == "EURO":
        cotacao = 5.22
    
    return valor_puro * cotacao

In [9]:
qtd_beneficio_real = []
qtd_beneficio_potencial = []
qtd_beneficio_qualitativo = []
valor_total_beneficios_reais = []
valor_total_beneficios_potenciais = []
descricao_beneficios = []

for index, demanda in demandas.iterrows():
    beneficios = beneficiosDemanda.loc[beneficiosDemanda.id_demanda == demanda.id_demanda]
    qtd_real = 0
    qtd_potencial = 0
    qtd_qualitativo = 0
    valor_total_real = 0.0
    valor_total_potencial = 0.0
    descricao_beneficios_demanda = ""
    
    for index, beneficio in beneficios.iterrows():
        tipo = beneficio.tipo_beneficio
        
        if tipo == "QUALITATIVO":
            qtd_qualitativo += 1
        elif tipo == "REAL":
            if np.isnan(beneficio.valor):
                continue
            
            qtd_real += 1
            valor_total_real += converter_valor(beneficio.valor, beneficio.moeda)
        elif tipo == "POTENCIAL":
            if np.isnan(beneficio.valor):
                continue
                
            qtd_potencial += 1
            valor_total_potencial += converter_valor(beneficio.valor, beneficio.moeda)
            
        descricao_beneficios_demanda += "\n " + beneficio.descricao
        
    
    qtd_beneficio_real.append(qtd_real)
    qtd_beneficio_potencial.append(qtd_potencial)
    qtd_beneficio_qualitativo.append(qtd_qualitativo)
    valor_total_beneficios_reais.append(valor_total_real)
    valor_total_beneficios_potenciais.append(valor_total_potencial)
    descricao_beneficios.append(descricao_beneficios_demanda)
    
    
demandas["qtd_beneficio_real"] = qtd_beneficio_real
demandas["qtd_beneficio_potencial"] = qtd_beneficio_potencial
demandas["qtd_beneficio_qualitativo"] = qtd_beneficio_qualitativo
demandas["valor_total_beneficios_reais (BRL)"] = valor_total_beneficios_reais
demandas["valor_total_beneficios_potenciais (BRL)"] = valor_total_beneficios_potenciais   
demandas["descricao_beneficios (BRL)"] = descricao_beneficios    
    
demandas.head()

,id_demanda,frequencia_uso,objetivo,score,situacao_atual,titulo_demanda,tipo_solicitante,cargo_solicitante,departamento_solicitante,nome_solicitante,setor_solicitante,qtd_beneficio_real,qtd_beneficio_potencial,qtd_beneficio_qualitativo,valor_total_beneficios_reais (BRL),valor_total_beneficios_potenciais (BRL),descricao_beneficios (BRL)
0,1,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",1.7,Duis aute irure dolor in reprehenderit in volu...,Demanda 01,Solicitante,Técnico,TI,Diego,Suporte Técnico,0,0,2,0.0,0.0,\n Sit amet porttitor eget dolor morbi non arc...
1,2,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",1.7,Duis aute irure dolor in reprehenderit in volu...,Demanda 02,Solicitante,Técnico,TI,Diego,Suporte Técnico,0,0,2,0.0,0.0,\n Sit amet porttitor eget dolor morbi non arc...
2,3,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",56.0,Duis aute irure dolor in reprehenderit in volu...,Demanda 03,Solicitante,Técnico,TI,Henrique,Administração de Redes e Sistemas Internos,0,1,1,0.0,78.0,\n Sit amet porttitor eget dolor morbi non arc...
3,4,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",56.0,Duis aute irure dolor in reprehenderit in volu...,Demanda 04,Solicitante,Técnico,TI,Henrique,Administração de Redes e Sistemas Internos,0,1,1,0.0,78.0,\n Sit amet porttitor eget dolor morbi non arc...
4,11,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",12.0,Duis aute irure dolor in reprehenderit in volu...,Demanda Teste Histórico 01,Solicitante,Analista de Dados,TI,Eduarda,Segurança Dital,0,0,0,0.0,0.0,


In [10]:
CCs

,id_centrocusto,nome_centro_custo,numero_centro_custo
0,1,Departamento de Engenharia de Sistemas,90100341
1,2,Departamento de Desenvolvimento de Software,8099230
2,3,Departamento de Segurança Digital,7088129


In [11]:
CCsDemanda.head()

,id_demanda,id_centrocusto
0,1,1
1,2,1
2,3,3
3,4,3
4,11,2


In [12]:
quantidade_CC_demanda = []
nomes_CC_demanda = []

for index, demanda in demandas.iterrows():
    centros_custo_demanda = CCsDemanda[CCsDemanda.id_demanda == demanda.id_demanda]
    quantidade_CCs = 0
    nomes_CC = ""
    
    for index, centro_custo in centros_custo_demanda.iterrows():
        quantidade_CCs += 1
        nome = CCs.loc[CCs.id_centrocusto == centro_custo.id_centrocusto]
        nomes_CC += "\n " + nome.at[nome.index[0], "nome_centro_custo"]
        
    quantidade_CC_demanda.append(quantidade_CCs)
    nomes_CC_demanda.append(nomes_CC)
    
demandas["quantidade_CC"] = quantidade_CC_demanda
demandas["nomes_CC"] = nomes_CC_demanda

demandas.head()

,id_demanda,frequencia_uso,objetivo,score,situacao_atual,titulo_demanda,tipo_solicitante,cargo_solicitante,departamento_solicitante,nome_solicitante,setor_solicitante,qtd_beneficio_real,qtd_beneficio_potencial,qtd_beneficio_qualitativo,valor_total_beneficios_reais (BRL),valor_total_beneficios_potenciais (BRL),descricao_beneficios (BRL),quantidade_CC,nomes_CC
0,1,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",1.7,Duis aute irure dolor in reprehenderit in volu...,Demanda 01,Solicitante,Técnico,TI,Diego,Suporte Técnico,0,0,2,0.0,0.0,\n Sit amet porttitor eget dolor morbi non arc...,1,\n Departamento de Engenharia de Sistemas
1,2,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",1.7,Duis aute irure dolor in reprehenderit in volu...,Demanda 02,Solicitante,Técnico,TI,Diego,Suporte Técnico,0,0,2,0.0,0.0,\n Sit amet porttitor eget dolor morbi non arc...,1,\n Departamento de Engenharia de Sistemas
2,3,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",56.0,Duis aute irure dolor in reprehenderit in volu...,Demanda 03,Solicitante,Técnico,TI,Henrique,Administração de Redes e Sistemas Internos,0,1,1,0.0,78.0,\n Sit amet porttitor eget dolor morbi non arc...,1,\n Departamento de Segurança Digital
3,4,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",56.0,Duis aute irure dolor in reprehenderit in volu...,Demanda 04,Solicitante,Técnico,TI,Henrique,Administração de Redes e Sistemas Internos,0,1,1,0.0,78.0,\n Sit amet porttitor eget dolor morbi non arc...,1,\n Departamento de Segurança Digital
4,11,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",12.0,Duis aute irure dolor in reprehenderit in volu...,Demanda Teste Histórico 01,Solicitante,Analista de Dados,TI,Eduarda,Segurança Dital,0,0,0,0.0,0.0,,1,\n Departamento de Desenvolvimento de Software


In [13]:
# values = {
#     "qtd_beneficio_real": 0, 
#     "qtd_beneficio_potencial": 0, 
#     "qtd_beneficio_qualitativo": 0, 
#     "valor_total_beneficios_reais (BRL)": 0,
#     "valor_total_beneficios_potenciais (BRL)": 0,
#     "quantidade_CC": 0,
#     "nomes_CC": ""
# }
# demandas.fillna(value=values, inplace=True)

In [14]:
demandas.head()

,id_demanda,frequencia_uso,objetivo,score,situacao_atual,titulo_demanda,tipo_solicitante,cargo_solicitante,departamento_solicitante,nome_solicitante,setor_solicitante,qtd_beneficio_real,qtd_beneficio_potencial,qtd_beneficio_qualitativo,valor_total_beneficios_reais (BRL),valor_total_beneficios_potenciais (BRL),descricao_beneficios (BRL),quantidade_CC,nomes_CC
0,1,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",1.7,Duis aute irure dolor in reprehenderit in volu...,Demanda 01,Solicitante,Técnico,TI,Diego,Suporte Técnico,0,0,2,0.0,0.0,\n Sit amet porttitor eget dolor morbi non arc...,1,\n Departamento de Engenharia de Sistemas
1,2,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",1.7,Duis aute irure dolor in reprehenderit in volu...,Demanda 02,Solicitante,Técnico,TI,Diego,Suporte Técnico,0,0,2,0.0,0.0,\n Sit amet porttitor eget dolor morbi non arc...,1,\n Departamento de Engenharia de Sistemas
2,3,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",56.0,Duis aute irure dolor in reprehenderit in volu...,Demanda 03,Solicitante,Técnico,TI,Henrique,Administração de Redes e Sistemas Internos,0,1,1,0.0,78.0,\n Sit amet porttitor eget dolor morbi non arc...,1,\n Departamento de Segurança Digital
3,4,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",56.0,Duis aute irure dolor in reprehenderit in volu...,Demanda 04,Solicitante,Técnico,TI,Henrique,Administração de Redes e Sistemas Internos,0,1,1,0.0,78.0,\n Sit amet porttitor eget dolor morbi non arc...,1,\n Departamento de Segurança Digital
4,11,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",12.0,Duis aute irure dolor in reprehenderit in volu...,Demanda Teste Histórico 01,Solicitante,Analista de Dados,TI,Eduarda,Segurança Dital,0,0,0,0.0,0.0,,1,\n Departamento de Desenvolvimento de Software
